# Gene Sequence Analysis for Protein Expression Prediction

This Jupyter Notebook presents the process of extracting relevant and significant features from a gene sequence with the ultimate goal of predicting the protein level of expression.


In [104]:
import pandas as pd
from Bio.SeqUtils import gc_fraction
from Bio.SeqUtils import molecular_weight
from Bio.Seq import Seq
import numpy as np


In [67]:
data = pd.read_excel('Known_dataset_challenge_2023.xlsx')
data.iloc[:, 2:6] = data.iloc[:, 2:6].apply(lambda x: x.str.replace("'", ''))

data['PA'] = pd.to_numeric(data['PA'])
orf = data['ORF']
utr3 = data["3'UTR"]
utr5 = data["Promoter+5'UTE"]

new_features_data = pd.DataFrame()

In [94]:
new_features_data['GC_content'] = orf.apply(lambda x: gc_fraction(x))
new_features_data['cys_prob'] = orf.apply(lambda x: x.count('C') / (len(x) / 3))

In [101]:
def lengths_ratio(seq1, seq2):
    ratio = len(seq2) / len(seq1)
    return ratio

utr3_ratio = data.apply(lambda row: lengths_ratio(row['ORF'], row["3'UTR"]), axis=1)
utr5_ratio = data.apply(lambda row: lengths_ratio(row['ORF'], row["Promoter+5'UTE"]), axis=1)
new_features_data['orf_utr3_ratio'] = utr3_ratio
new_features_data['orf_utr5_ratio'] = utr5_ratio


In [70]:
def count_dinucleotides(seq):
    counts = {
        "AA": 0,
        "CC": 0,
        "AT": 0,
        "CA": 0,
        "CG": 0
    }

    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i + 2]
        if dinucleotide in counts:
            counts[dinucleotide] += 1
    
    return counts

dinucleotide_counts = orf.apply(count_dinucleotides).apply(pd.Series)
new_features_data = pd.concat([new_features_data, dinucleotide_counts], axis=1)


In [93]:
def calc_codons_prob(DNAsequence):
    codons_count = [0] * len(codons)
    
    for i in range(0, len(DNAsequence) - 2, 3):  
        curr_codon = DNAsequence[i:i + 3]
        for j, codon in enumerate(codons):  
            if curr_codon == codon:
                codons_count[j] += 1
                
    codons_prob = [count / (len(DNAsequence) / 3) for count in codons_count]
    
    return codons_prob

codons_prob = orf.apply(calc_codons_prob).apply(pd.Series)
new_features_data = pd.concat([new_features_data, codons_prob], axis=1)


In [91]:
amino_acid_to_int = {
    'A': 1,
    'R': 2,
    'N': 3,
    'D': 4,
    'C': 5,
    'Q': 6,
    'E': 7,
    'G': 8,
    'H': 9,
    'I': 10,
    'L': 11,
    'K': 12,
    'M': 13,
    'F': 14,
    'P': 15,
    'S': 16,
    'T': 17,
    'W': 18,
    'Y': 19,
    'V': 20
}

codons = ['TTT', 'TTC', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG', 'ATT', 'ATC', 'ATA', 'ATG',
            'GTT', 'GTC', 'GTA', 'GTG', 'TCT', 'TCC', 'TCA', 'TCG', 'CCT', 'CCC', 'CCA', 'CCG',
            'ACT', 'ACC', 'ACA', 'ACG', 'GCT', 'GCC', 'GCA', 'GCG', 'TAT', 'TAC', 'TAA', 'TAG',
            'CAT', 'CAC', 'CAA', 'CAG', 'AAT', 'AAC', 'AAA', 'AAG', 'GAT', 'GAC', 'GAA', 'GAG',
            'TGT', 'TGC', 'TGA', 'TGG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGT', 'AGC', 'AGA', 'AGG',
            'GGT', 'GGC', 'GGA', 'GGG']


In [103]:
def find_most_frequent_aa(seq):
    dna_seq = Seq(seq)
    aa_seq = dna_seq.translate()
    letter_count = {}
    for letter in aa_seq:
        if letter in letter_count:
            letter_count[letter] += 1
        else:
            letter_count[letter] = 1

    max_count = max(letter_count.values())
    most_frequent_letters = [letter for letter, count in letter_count.items() if count == max_count]
    amino_acid_int = amino_acid_to_int.get(most_frequent_letters[0])
    return amino_acid_int

new_features_data['most_freq_AA'] = orf.apply(find_most_frequent_aa).apply(pd.Series)

In [111]:
def find_PSSM(ORF, UTR5):
    def slice_last_6(x):
        return x[-6:]

    def slice_first_3(x):
        return x[0:3]

    sliced_utr5 = [slice_last_6(seq) for seq in UTR5]
    sliced_orf = [slice_first_3(seq) for seq in ORF]

    window = [utr + orf for utr, orf in zip(sliced_utr5, sliced_orf)]

    mat_positions_nuc = np.zeros((4, 9))

    for position in range(9):
        current_position = [seq[position] for seq in window]
        a_count = current_position.count('A')
        t_count = current_position.count('T')
        c_count = current_position.count('C')
        g_count = current_position.count('G')
        mat_positions_nuc[0, position] = a_count
        mat_positions_nuc[1, position] = t_count
        mat_positions_nuc[2, position] = c_count
        mat_positions_nuc[3, position] = g_count

    sum_columns = np.sum(mat_positions_nuc, axis=0)
    frequencies = mat_positions_nuc / sum_columns

    PSSM = pd.DataFrame(frequencies.T, columns=['A', 'T', 'C', 'G'],
                        index=[f'position{str(i)}' for i in range(1, 10)])

    return PSSM, window



['TTTCAGATG', 'ATTGTGATG', 'ATTCTAATG', 'AGTTTAATG', 'CGAAAAATG', 'CTGAAAATG', 'CACTTGATG', 'GCCAATATG', 'TTCAGAATG', 'TTCCTGATG', 'TGCCGCATG', 'GTAACCATG', 'AGCCTAATG', 'TTCAGTATG', 'AAAATTATG', 'GAATTAATG', 'CACTTGATG', 'AAATTGATG', 'TTTCCAATG', 'AATGTGATG', 'AACATCATG', 'AAATTAATG', 'GTTCAAATG', 'GGAATAATG', 'TTTCTAATG', 'TCCATAATG', 'AAATAAATG', 'AGATTAATG', 'CGTTTGATG', 'ACAATAATG', 'AACATTATG', 'AACATGATG', 'GGAATGATG', 'TGAGAGATG', 'CTAGATATG', 'CCGATGATG', 'GCAGAGATG', 'CAAGCCATG', 'TAAACCATG', 'AGAAACATG', 'GCTTTAATG', 'AGATTAATG', 'AAAACAATG', 'ATCCCGATG', 'AAAGTGATG', 'AAAAATATG', 'TTCCTAATG', 'ATGTTGATG', 'TTCAGAATG', 'TCAAAAATG', 'CTAGAAATG', 'ATTTTGATG', 'GGGATGATG', 'CAACCGATG', 'AATCAGATG', 'TACCCCATG', 'CCAGAGATG', 'CAGTTTATG', 'TGCTTAATG', 'GTTGGAATG', 'TTGTTAATG', 'TTATTAATG', 'GGAGTAATG', 'TATGTAATG', 'TTCCCAATG', 'ATCGTTATG', 'TAATTAATG', 'TGCACAATG', 'TTAAGGATG', 'AATTTAATG', 'AGCCTGATG', 'TTATATATG', 'ACCAAAATG', 'CCCGAAATG', 'ATCGAGATG', 'TTTGTGATG', 'TGAAAGATG'

In [137]:
def find_ATG_context_score(orf, utr5):
    pssm, window = find_PSSM(orf, utr5)
    scores = []
    for row in window:       
        atg_context_score = 1.0
        for position in range(9):
            nuc = row[position]
            prob = pssm[nuc][position]
            atg_context_score *= prob
        scores.append(atg_context_score)
    return scores

new_features_data['ATG_context_score'] = find_ATG_context_score(orf, utr5)

In [139]:
new_features_data['molecular_weight'] = orf.apply(molecular_weight)

In [ ]:
new_features_data.to_excel('new_known_dataset1.xlsx', index=False)